## Worksheet 10

### Group 22 Eaton Heidegger and Shannon Schröder

In [4]:
import numpy as np
from numba import njit, jit


a) Generate initial conditions for N = 105 particles by randomly sampling from this distribution
 function using either Box-Muller or `numpy.random.randn`.

We are using `numpy.random.randn`, which already gives standard-normal deviates, so no Box-Muller is required.

In [ ]:
# --- parameters ------------------------------------------------------------
N       = 100_000                         # number of particles
sigma_x = 0.15
sigma_y = 0.10
m       = 1.0 / N                         # particle mass
sigma_p = m * 1e-4                        # momentum-space width (very narrow!)

# --- sample configuration space -------------------------------------------
# draw from N(0.5, σ²) for x and y, then wrap into [0,1)
X = np.empty((N, 2))
X[:, 0] = 0.5 + sigma_x * np.random.randn(N)   # x–coordinate
X[:, 1] = 0.5 + sigma_y * np.random.randn(N)   # y–coordinate
X = np.fmod(X, 1.0)                     # periodic domain [0,1)²

# --- sample momentum space -------------------------------------------------
P = np.empty_like(X)
P[:]  = sigma_p * np.random.randn(N, 2)        # (p_x , p_y)

# X and P are now both shape (N, 2).  Each particle has mass m = 1/N.
print(X.shape, P.shape, m, sigma_p)


(100000, 2) (100000, 2) 1e-05 1e-09


Here, we: 
- Draw $x$ and $y$ independently from $\mathcal{N}(0.5,\sigma_x^2)$ and $\mathcal{N}(0.5,\sigma_y^2)$.
- `np.fmod` folds any values outside the unit square back into the periodic domain for the position sampling. 

For the momentum sampling, we
* Draw $p_x$ and $p_y$ from $\mathcal{N}(0,\sigma_p^2)$ with the very small width $\sigma_p = m\,10^{-4}$.

**Outputs**

   * `X` and `P` are ready for the spectral-PIC loop.
   * Each particle’s mass is fixed at $m = 1/N$, so we don’t need a separate mass array.


b) Use the function `cic_deposit` from the lecture notes to interpolate the positions X to a grid of resolution M × M with M = 64 with weights m to obtain the density field ρ.

 Display ρ
 as an image. Compare it to the true initial density which is obtained by integrating f0 over
 momentum space (generate another image corresponding to it).

In [ ]:
# CIC implementation from lecture
@jit(nopython = True)
def cic_deposit(X, Y, W, ngrid):
    """Deposit particle poitions X, Y with weights W onto 2D grid 
    of size ngrid x ngrid.
    X, Y: 1D arrays of particle positions in [0,1)
    W: 1D array of particle weights (e.g. mass * f0)
    ngrid: number of grid cells in each dimenstion
    """
    rho = np.zeros((ngrid, ngrid))
    for x,y,w in zip(X, Y, W):
        x = np.fmod(1.0 +x,1.0)
        y = np.fmod(1.0+y,1.0)

        il = int(np.floor(x*ngrid))
        ir = (il+1) %ngrid
        jl = int(np.floor(y*ngrid))
        jr = (jl+q) %ngrid

        dx = x*ngrid-float(il)
        dy = y*ngrid-fleat(jl)

        rho[il, jl] += (1-dx) * (1-dy) * W
        rho[il ]





    